In [7]:
import streamlit as st
import smtplib
from PIL import Image
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import numpy as np
import pandas as pd #-> Để update data dạng bản
import json 

import matplotlib.image as mpimg
from google.oauth2 import service_account
from gsheetsdb import connect
from datetime import datetime, timedelta
from datetime import datetime as dt
from typing import Text
from numpy.core.numeric import NaN
import streamlit as st
import json
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from streamlit_pandas_profiling import st_profile_report


In [11]:

# Initial setup
st.set_page_config(layout="wide")

def get_df(file):
  # get extension and read file
  extension = file.name.split('.')[1]
  if extension.upper() == 'CSV':
    df = pd.read_csv(file)
  elif extension.upper() == 'XLSX':
    df = pd.read_excel(file, engine='openpyxl')
  elif extension.upper() == 'PICKLE':
    df = pd.read_pickle(file)
  return df


In [12]:
# Create a connection object.
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"],
    scopes=['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive'],
)
gc1 = gspread.authorize(credentials)
spreadsheet_key = '1_APNLrt6uNo6aaZIERHYAssD_KEU2pEqEnInG1ewnWY' # input SPREADSHEET_KEY HERE

gc2=gspread.authorize(credentials)
spreadsheet_key='1Kf79UeBTa0q2NAh4PaW2Y1nqE__S0wiSQSOkk2dkQm0'
#email_list
sh1=gc1.open("Group 8 - Atom Fair - Dataset").worksheet('Email list')
data1=sh1.get_all_records()
email_list=pd.DataFrame(data1)
email_list=email_list.replace("",np.nan)
email_list.columns=email_list.columns.str.replace(" ","_")
email_list['Student_ID'].astype(str)

#td_df
sh2=gc1.open("Group 8 - Atom Fair - Dataset").worksheet('Results')
data=sh2.get_all_records()
df=pd.DataFrame(data)
df=df.replace((0,np.nan))
df.columns=data_df.columns.str.replace(" ","_")
df

,Student_ID,Intake,NOTE,OBRW,OVWL,OFIN,OMAT,OSTA,OWIN,OREC,...,BMIK,BMGT,BACC,PFIN,PACC,BWET,BMAK,PMAR,PWIN,BSEM
0,315820,BA2018,,50,45,55,65,65,45,60,...,50,50,45,0,0,0,0,0,0,0
1,315577,BA2018,,55,45,55,55,55,45,60,...,50,50,45,0,0,0,0,0,0,0
2,315526,BA2018,,75,80,65,75,85,75,85,...,85,75,75,0,0,0,0,0,90,0
3,315514,BA2018,,85,85,100,90,90,60,85,...,50,85,65,0,0,0,0,0,85,0
4,315454,BA2018,,60,60,45,50,50,45,75,...,50,85,45,0,0,0,0,0,85,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,310132,FA2015,Terminated,65,45,50,65,55,45,65,...,55,65,50,85,85,75,45,80,70,85
298,311078,FA2015,Terminated,55,55,55,90,70,70,70,...,60,80,80,45,75,65,45,80,70,85
299,39591,FA2015,Terminated,65,50,55,80,60,70,70,...,65,55,55,45,65,55,50,80,70,85
300,310474,FA2015,Terminated,50,50,65,60,50,50,70,...,45,50,55,45,65,50,45,80,70,85


In [13]:

def transform(df):
  # SUMMARY
    df_types = pd.DataFrame(df.dtypes, columns=['Data Type'])
    numerical_cols = df_types[~df_types['Data Type'].isin(['object',
                   'bool'])].index.values
    df_types = pd.DataFrame(df.dtypes, columns=['Data Type'])
    numerical_cols = df_types[~df_types['Data Type'].isin(['object',
                   'bool'])].index.values
    first_cols=st.multiselect('First academic Object names',
                        numerical_cols.tolist(),
                        numerical_cols.tolist())
    second_cols=st.multiselect('Second academic Object names',
                        df.columns.tolist(),
                        df.columns.tolist())
    _pass=st.text_input('Enter mark to pass exam:')
    first_df=df[first_cols]

    # first_pass=_pass.loc[_pass.str.startswith('O')].groupby('Intake').count()

    # df2=df[first_cols]
    # df3=df[second_cols]
    return first_df
transform(df)





In [ ]:

def explore(df):
    
    _pass=st.text_input('Enter mark to pass exam:')

    O=_pass.loc[_pass.Object.str.startswith('O')].groupby('Intake').count()


_passO=O.reset_index()
l=_pass.groupby(['Intake','Student_ID']).count()
l=l.reset_index()
_pass18=l.loc[l.Scores>=18]
st.markdown('### SỐ LƯỢNG SINH VIÊN ĐẬU MÔN O THEO LỚP')
fig, ax = plt.subplots()
sns.barplot(data=_passO,x="Intake",y="Scores")
st.pyplot(fig)
st.markdown("")
st.markdown("### SỐ LƯỢNG SINH VIÊN ĐÃ HỌC XONG 18 MÔN O,B,P")
sns.barplot(data=_pass18,x="Intake",y="Scores")
st.pyplot(fig)





In [ ]:
def main():
    st.title('Student a dataset')
    file1 = st.file_uploader("Upload file", type=['csv','xlsx','pickle'])
    if not file1:
        st.write("Upload a .csv or .xlsx file to get started")
    else:
        df = get_df(file1)
        data_df=df.replace((0,np.nan))
        data_df.columns=data_df.columns.str.replace(" ","_")
        list_cols = transform(df)
        st.write(data_df)
        explore(df)
        

main()